<a href="https://colab.research.google.com/github/brostromb/ai-ml-principles-exercises/blob/main/ML-training/logging-trainings-with-WandB/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install dependencies and import packages
First we need to install the libraries we will be using. We will use `numpy` for generic matrix operations and `tensorflow` for deep learning operations such as convolutions, pooling and training (backpropagation).

In [1]:
import sys
!{sys.executable} -m pip install numpy tensorflow
from IPython.display import clear_output
clear_output()
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [7]:
from IPython.display import clear_output

!pip install wandb

clear_output()

In [10]:
import wandb
wandb.login()

wandb: Currently logged in as: bjorn-brostrom. Use `wandb login --relogin` to force relogin


True

## Create a neural network
Next we define a function that can be used to build a neural network. The neural network is a simple CNN (convolutional neural network) used for classification. The structure of the network is not important for this exercise, you can instead see it as a black box that can be trained to classify an input image.

In [2]:
def create_cnn(input_shape, output_classes):
    return keras.Sequential(
        [
            keras.Input(shape=input_shape),
            layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Flatten(),
            layers.Dense(output_classes, activation="softmax"),
        ]
    )

## Prepare the data
The neural network will be trained on a digit classification dataset called *MNIST*. This code downloads and loads the images together with their true labels. The code also does some preprocessing of the data to make it more suitable for a neural network.

In [3]:
def get_mnist_data():
    # Load the data and split it between train and test sets
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

    # Scale images to the [0, 1] range
    x_train = x_train.astype("float32") / 255
    x_test = x_test.astype("float32") / 255

    # Make sure images have shape (28, 28, 1)
    x_train = np.expand_dims(x_train, -1)
    x_test = np.expand_dims(x_test, -1)

    # convert class vectors to binary class matrices
    num_classes = 10
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    return (x_train, y_train), (x_test, y_test)

## Train the network
Finally we will train the network on the data to teach it how to classify a digit. We create a model which expects a 28x28 pixel monocolor image since this is the format the images in the *MNIST* dataset are. We then create an optimizer and calls the `fit()` method to start the training.

In [16]:
batch_size = 128
epochs = 3


In [5]:
# Get the training data
x_train, y_train = get_mnist_data()[0]

11490434/11490434 [==============================] - 0s 0us/step


In [17]:
#skapar ny mätning
wandb.init(project="ai-ml-exercise", config={"batch_size": batch_size, "epochs": epochs})
# Create a Convolutional Neural Network that
# expects a 28x28 pixel image with 1 color chanel (gray) as input
model = create_cnn((28, 28, 1), 10)

model.compile(loss="categorical_crossentropy",
              optimizer="adam", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=batch_size,
          epochs=epochs, validation_split=0.1,
          callbacks=[wandb.keras.WandbCallback()])

accuracy,▁▆▇▇▇▁▆▇▇▇▇▇████████
epoch,▁▁▂▃▃▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▃▂▂▂█▃▂▂▂▂▁▁▁▁▁▁▁▁▁
val_accuracy,▁▄▆▇▇▂▄▆▆▇▇▇▇█▇▇██▇▇
val_loss,█▄▃▂▂▇▄▃▂▂▁▁▂▁▁▂▁▂▂▂
accuracy,0.99719
best_epoch,11
best_val_loss,0.03481
epoch,14
loss,0.00914
val_accuracy,0.99033


Epoch 1/3
422/422 [==============================] - ETA: 0s - loss: 0.2956 - accuracy: 0.9157

wandb: Adding directory to artifact (/content/wandb/run-20221206_133221-fmhejnde/files/model-best)... Done. 0.0s


422/422 [==============================] - 46s 108ms/step - loss: 0.2956 - accuracy: 0.9157 - val_loss: 0.0862 - val_accuracy: 0.9780
Epoch 2/3
422/422 [==============================] - ETA: 0s - loss: 0.0813 - accuracy: 0.9755

wandb: Adding directory to artifact (/content/wandb/run-20221206_133221-fmhejnde/files/model-best)... Done. 0.0s


422/422 [==============================] - 46s 109ms/step - loss: 0.0813 - accuracy: 0.9755 - val_loss: 0.0667 - val_accuracy: 0.9805
Epoch 3/3
422/422 [==============================] - ETA: 0s - loss: 0.0600 - accuracy: 0.9815

wandb: Adding directory to artifact (/content/wandb/run-20221206_133221-fmhejnde/files/model-best)... Done. 0.0s


422/422 [==============================] - 45s 107ms/step - loss: 0.0600 - accuracy: 0.9815 - val_loss: 0.0497 - val_accuracy: 0.9868
